In [3]:
from pathlib import Path
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI


PASTA_ARQUIVOS = Path('/Users/marcelogalli/Documents/GitHub/IA-Langchain-Example/projeto1/arquivos').parent / 'arquivos'
MODEL_NAME = 'gpt-4o-mini'


def importacao_arquivos():
    documentos = []
    for arquivo in PASTA_ARQUIVOS.glob("*.pdf"):
        loader = PyPDFLoader(str(arquivo))
        documentos_arquivo = loader.load()
        # adicionando os elementos da primeira lista na segunda
        documentos.extend(documentos_arquivo)
    return documentos


def split_de_documentos(documentos):
    recursive_spliter = RecursiveCharacterTextSplitter(
        chunk_size=2500,
        chunk_overlap=250,
        separators=["\n\n", "\n", " ", ""]
    )

    documentos = recursive_spliter.split_documents(documentos)

    for i, doc in enumerate(documentos):
        doc.metadata['source'] = doc.metadata['source'].split('/')[1]
        doc.metadata['doc_id'] = i

    return documentos


def cria_vector_store(documentos):
    embedding_model = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(
        documents=documentos,
        embedding=embedding_model
    )
    return vector_store


def cria_chain_conversa(vector_store):
    chat = ChatOpenAI(
        model=MODEL_NAME
    )
    memory = ConversationBufferMemory(
        return_messages=True,
        memory_key='chat_history',
        output_key='answer'
    )
    retriever = vector_store.as_retriever()
    chat_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        memory=memory,
        retriever=retriever,
        return_source_documents=True,
        verbose=True
    )

    return chat_chain


In [4]:
documentos = importacao_arquivos()
documentos = split_de_documentos(documentos)
vector_store = cria_vector_store(documentos)
chain = cria_chain_conversa(vector_store)

/var/folders/v8/mwd2w9sx4vscpt3ttqlvwws40000gn/T/ipykernel_7604/3607096731.py:54: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [6]:
chain.invoke({'question': 'quem é o outorgante dessa procuração?'})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: quem é o outorgante dessa procuração?
Assistant: O outorgante da procuração é JEANNE CORREIA DE MELLO DA GLORIA.
Follow Up Input: quem é o outorgante dessa procuração?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
PROCURAÇÃO PÚBLICA_Carina Home 501 
Pelo presente instrumento par�cular de procuração, eu, JEANNE CORREIA DE MELLO DA GLORIA, 
brasileira, portadora do RG nº 38.986.986-7 e inscrita no CPF sob o nº 718.758.791-02, atualmente 
residente e domiciliada em 14209 Cascadian Way

{'question': 'quem é o outorgante dessa procuração?',
 'chat_history': [HumanMessage(content='quem é o outorgante dessa procuração?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='O outorgante da procuração é JEANNE CORREIA DE MELLO DA GLORIA.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quem é o outorgante dessa procuração?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='O outorgante da procuração é JEANNE CORREIA DE MELLO DA GLORIA.', additional_kwargs={}, response_metadata={})],
 'answer': 'O outorgante da procuração é JEANNE CORREIA DE MELLO DA GLORIA.',
 'source_documents': [Document(metadata={'source': 'Users', 'page': 0, 'doc_id': 0}, page_content='PROCURAÇÃO PÚBLICA_Carina Home 501 \nPelo presente instrumento par�cular de procuração, eu, JEANNE CORREIA DE MELLO DA GLORIA, \nbrasileira, portadora do RG nº 38.986.986-7 e inscrita no CPF sob o nº 718.758.791-02, atualmente \nresidente e domiciliada em 14209 Cascadia